In [4]:
import pandas as pd
import numpy as np
from os.path import dirname, abspath


In [7]:
d = dirname(dirname(abspath("preprocessing.ipynb")))
df = pd.read_csv(d + '/data/Semi_processed_data.csv')

feature_names = ['start station name', 'end station name', 'gender',
                 'tripduration','start station longitude', 'end station longitude',
                 'start station latitude','end station latitude','Start Time',
                 'End Time','birth year']

label = 'usertype'


In [8]:
df.head()

,tripduration,start station name,start station latitude,start station longitude,end station name,end station latitude,end station longitude,usertype,birth year,gender,Start Time,End Time
0,142,Exchange Place,40.716247,-74.033459,Harborside,40.719252,-74.034234,Subscriber,1963.0,1,56100.518,56242.572
1,223,Exchange Place,40.716247,-74.033459,Grand St,40.715178,-74.037683,Subscriber,1992.0,2,61245.326,61468.986
2,106,Exchange Place,40.716247,-74.033459,Paulus Hook,40.714145,-74.033552,Subscriber,1960.0,1,61679.762,61785.876
3,370,Exchange Place,40.716247,-74.033459,Newark Ave,40.721525,-74.046305,Subscriber,1976.0,1,61769.646,62140.091
4,315,Exchange Place,40.716247,-74.033459,Manila & 1st,40.721651,-74.042884,Subscriber,1980.0,1,62391.685,62707.576


In [9]:
X = df.loc[:,df.columns != label].values
y = df.loc[:,df.columns == label].values

In [10]:
df_y = df.drop(columns=feature_names)


In [12]:
#Balance
balance = df_y['usertype'].value_counts(normalize = True)
balance

Subscriber    0.92453
Customer      0.07547
Name: usertype, dtype: float64

In [13]:
base = balance[0]

In [14]:
print("balance of the data / baseline accuracy = ",base)

balance of the data / baseline accuracy =  0.9245295186804018


* **ML pipeline with logistic regression**

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib
import statistics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.compose import ColumnTransformer
import matplotlib
from matplotlib import pylab as plt
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")


In [21]:
def ML_pipeline_kfold_GridSearchCV_log(X,y,random_state,n_folds):
    # create a test set
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state,stratify=y)
    # splitter for _other
    kf = StratifiedKFold(n_splits=n_folds,shuffle=True,random_state=random_state)
    # create the pipeline: preprocessor + supervised ML method
    
    feature_names = ['start station name', 'end station name', 'gender',
                     'tripduration','start station longitude', 'end station longitude',
                     'start station latitude','end station latitude','Start Time',
                     'End Time','birth year']

    cat_ftrs = ['start station name', 'end station name', 'gender']
    num_ftrs = ['tripduration','start station longitude', 'end station longitude','start station latitude',
                'end station latitude','Start Time','End Time','birth year']
    
    cat_ftrs_i = [df.columns.get_loc(x) for x in cat_ftrs]
    num_ftrs_i = [df.columns.get_loc(x) for x in num_ftrs]

    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(sparse=False))])
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])


    # collect the transformers
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_ftrs_i),
            ('cat', categorical_transformer, cat_ftrs_i)]) 

    pipe = make_pipeline(preprocessor, LogisticRegression(penalty='11', solver='saga',
                                                          max_iter=10000, multi_class = 'auto', random_state = 22))
    #pipe = make_pipeline(preprocess, LogisticRegression(penalty='l1', solver='saga', max_iter=10000))
    
    # the parameter(s) we want to tune
    param_grid = {'logisticregression__C': [0.1, 1.0, 10, 100]}
    # prepare gridsearch
    grid = GridSearchCV(pipe, param_grid=param_grid,scoring = make_scorer(accuracy_score),
                            cv=kf, return_train_score = True,iid=True)
    # do kfold CV on _other
    grid.fit(X_other, y_other)
    return grid, grid.score(X_test, y_test)



In [22]:
test_scores = []

for i in range(10):
    grid, test_score = ML_pipeline_kfold_GridSearchCV_log(X,y.ravel(),i*22, 5)
    print(grid.best_params_)
    print('best CV score:',grid.best_score_)
    print('test score:',test_score)
    test_scores.append(test_score)
    
print('test accuracy:',np.around(np.mean(test_scores),2),'+/-',np.around(np.std(test_scores),2))

mean_LR_a = np.mean(test_scores)
mean_LR_sd = np.std(test_scores)

IndexError: index 11 is out of bounds for axis 0 with size 11